**Bibliotecas utilizadas**

In [200]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

**Criando Dataframe - COVID, FLU, COLD Symptoms**

In [201]:
URL = 'https://janeawsdata.s3.us-east-2.amazonaws.com/large_data.csv'
df = pd.read_csv(URL)

**Mapeando as classes para valores numéricos**


In [202]:
mappings = {
   'ALLERGY': 0,
   'COLD': 1,
   'COVID': 2,
   'FLU': 3
}
df['TYPE'] = df['TYPE'].apply(lambda x: mappings[x])

**Utilizando a função train_test_split para dividir os dados em 80/20 % treino e teste**

In [203]:
X = df.drop('TYPE', axis=1).values
y = df['TYPE'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convertendo os dados para PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


**Definindo a Rede Neural**

In [204]:
class ANN(nn.Module):
   def __init__(self):
       super().__init__()
       # número de neurônios das camadas foi escolhido de forma arbitrária
       self.fc1 = nn.Linear(in_features=20, out_features=40) #Entrada = 20 (features em x)
       self.fc2 = nn.Linear(in_features=40, out_features=80)
       self.fc3 = nn.Linear(in_features=80, out_features=40)
       self.output = nn.Linear(in_features=40, out_features=4) #Saída = 4 (classes)
 
   # função forward que será responsável pela propagação da rede
   def forward(self, x):
     x = F.relu(self.fc1(x)) #função relu para retornar sempre valores positivos
     x = F.relu(self.fc2(x))
     x = F.relu(self.fc3(x))
     x = self.output(x)
     return x

**Instânciando o modelo**

In [205]:
model = ANN()

In [206]:
model.output

Linear(in_features=40, out_features=4, bias=True)

**Critério - CrossEntropyLoss e Algoritmo de otimização - Adam**



In [207]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1) #algoritmo e taxa de aprendizagem do optimizer

**Treinamento de modelo - 100 épocas**

In [208]:
epochs = 100 
loss_arr = []
for i in range(epochs):
   y_hat = model.forward(X_train)
   loss = criterion(y_hat, y_train)
   loss_arr.append(loss)
 
   if i % 10 == 0:
       print(f'Epocas: {i} Perda: {loss}')
 
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

Epocas: 0 Perda: 1.3648980855941772
Epocas: 10 Perda: 0.33289963006973267
Epocas: 20 Perda: 0.23538248240947723
Epocas: 30 Perda: 0.20973087847232819
Epocas: 40 Perda: 0.19235682487487793
Epocas: 50 Perda: 0.15728305280208588
Epocas: 60 Perda: 0.10982885211706161
Epocas: 70 Perda: 0.12121178954839706
Epocas: 80 Perda: 0.11022529751062393
Epocas: 90 Perda: 0.1049908995628357


**Avaliando o modelo**

In [209]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [210]:
dfcorrecao = pd.DataFrame({'Y': y_test, 'YHat': preds})
dfcorrecao['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(dfcorrecao['Y'], dfcorrecao['YHat'])] #compara o valor de Y com o YHat valor estimado

In [211]:
print(classification_report(y_test, preds, df.TYPE.unique()))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      3177
           1       0.54      0.90      0.67       233
           2       0.49      0.84      0.62       419
           3       0.98      0.92      0.95      5062

    accuracy                           0.93      8891
   macro avg       0.75      0.90      0.81      8891
weighted avg       0.95      0.93      0.94      8891

